In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableMap
from langchain_core.output_parsers import StrOutputParser
from typing import List

LLM과 Retriever를 받아 LangChain RAG 체인을 생성합니다.

Args:
    llm: LLM 모델 (예: ChatOpenAI)
    retriever: 벡터스토어에서 만든 retriever 객체

Returns:
    Runnable 체인 객체 (질문에 대해 RAG 응답을 생성함)

In [ ]:
# 요약 프롬프트
summary_prompt = ChatPromptTemplate.from_template("""
다음 문서를 3문장 이내로 요약해 주세요:

문서:
{document}
""")

# 요약 함수
def summarize_documents(llm, docs: List):
    summarized = []
    for doc in docs:
        prompt = summary_prompt.format(document=doc.page_content)
        summary = llm.invoke(prompt)
        summarized.append(summary)
    return summarized

In [ ]:
# 필터링 함수 예시: 키워드 기반 필터링
def filter_documents_by_keyword(docs: List, keyword: str):
    return [doc for doc in docs if keyword.lower() in doc.page_content.lower()]

In [ ]:
# 최종 RAG 체인 생성
def create_custom_rag_with_summary_filter(llm, retriever, keyword: str):
    # 질의응답 프롬프트
    rag_prompt = ChatPromptTemplate.from_template("""
    다음은 질문에 관련된 문서 요약입니다:

    요약된 문서:
    {context}

    질문:
    {question}

    답변:
    """)

    def get_context(input):
        # 1. 관련 문서 검색
        docs = retriever.get_relevant_documents(input["question"])
        # 2. 키워드 필터링
        filtered_docs = filter_documents_by_keyword(docs, keyword)
        # 3. 요약
        summaries = summarize_documents(llm, filtered_docs)
        return {
            "context": "\n\n".join(summaries),
            "question": input["question"]
        }

    rag_chain = (
        RunnableMap(get_context)
        | rag_prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain

입력: {"question": "AI 기술은 무엇인가요?"}
   ↓
get_context (RunnableMap)
   ↓
{context: "...", question: "AI 기술은..."}
   ↓
rag_prompt.format(...)
   ↓
LLM에 프롬프트 전달
   ↓
LLM 응답 (ChatMessage 형태)
   ↓
StrOutputParser() → 최종 답변 문자열

In [ ]:
llm = ChatOpenAI(model="gpt-4.1-mini")  # or gpt-4o
retriever = vectorstore.as_retriever()
keyword = "제안요청서"  # 예시: "제안요청서"가 포함된 문서만 사용

rag_chain = create_custom_rag_with_summary_filter(llm, retriever, keyword)

response = rag_chain.invoke({"question": "?"})
print(response)

1. 사용자 질문이 {"question": ""} 형태로 입력됨
2. RunnableMap(get_context)가 작동해서 retriever가 관련 문서들을 검색
3. filter_documents_by_keyword가 문서 중 키워드(예: "제안요청서") 포함된 문서만 골라냄
4. summarize_documents가 각 문서를 3문장 이내로 요약
5. 여러 요약문을 하나의 문자열로 합침
최종적으로 { "context": "요약된 문서 내용들", "question": "?" } 딕셔너리 반환